In [17]:
import pandas as pd #Se importa la libreria pandas 
df=pd.read_csv('calles_de_medellin_con_acoso.csv', delimiter = ';') #Se lee el data frame 
df['name'] = df['name'].str.lower()   #Se pone en minúsucula todas las filas de la columna 'name'
df["harassmentRisk"]=df["harassmentRisk"].interpolate()  #Interpolar en la columna harassmentRisk para llenar los datos que sean NaN
df['peso1'] = (df["length"]**(2*df["harassmentRisk"]))   #Hago una nueva columna que combine el length y harrassmentRisk (variable v), que luego será el peso de cada calle
df['peso2'] = (df["length"]+(100*df["harassmentRisk"]))
df['peso3'] = (df["length"]*(df["harassmentRisk"]))
df=df.dropna() #Elimino las filas que tengan algún dato nulo, en este caso en la columna 'name'

In [18]:
origenes=list(df["origin"]) #Lista con todos los origenes
destinos=list(df["destination"]) #Lista con todos los destinos

In [19]:
for element in destinos: #Para cada elemento en destinos
    if element not in origenes: #Si el elemento no está en origenes
        origenes.append(element) #Lo agrego

In [20]:
conexiones=[] # Lista que contendra las conexiones de cada uno de los elementos en origenes
pesos1=[] #Lista que contendra el peso de las conexiones de cada uno de los elementos en origenes
pesos2=[]
pesos3=[]
for origen in origenes: #Para cada origen en origenes
    c=list(df.query("origin==@origen")["destination"]) #Filtro el dataset y hago una lista de los destinos que tienen como origin el origen
    p1=list(df.query("origin==@origen")["peso1"]) #Filtro el dataset y hago una lista de los pesos que tienen como origin el origen
    p2=list(df.query("origin==@origen")["peso2"])
    p3=list(df.query("origin==@origen")["peso3"])
    conexiones.append(c) #Agrego las conexiones a la lista
    pesos1.append(p1) #Agrego los pesos a la lista
    pesos2.append(p2)
    pesos3.append(p3)

In [21]:
'''
Cada elemnto en conexiones y en pesos es una lista que contiene estos datos para cada elemento de origenes
Al comparar sus longitudes, verificamos que a cada elemento en origenes le corresponda un elemento en conexiones y uno en pesos
'''
len(origenes)==len(conexiones)==len(pesos1)==len(pesos2)==len(pesos3)

True

In [22]:
diccionario1={} #Creo la lista de adyacencia
diccionario2={}
diccionario3={}

for i in range(len(origenes)): #Para cada posición en origenes,conexiones y pesos
    diccionario1[origenes[i]]=[conexiones[i],pesos1[i]] #hago la posición de origenes la llave, la de conexiones la primera lista de los valores y la de pesos la segunda
    diccionario2[origenes[i]]=[conexiones[i],pesos2[i]]
    diccionario3[origenes[i]]=[conexiones[i],pesos3[i]]

In [23]:
'''Esta clase define los vértices del grafo'''
class Vertice:

    def __init__ (self, v):   #Defino el constructor de la clase vértice
        self.id = v   #Instancio la etiqueta del vértice
        self.adyacentes = []   #Instancio la lista de los vertices adyacentes a v
        self.visitado = False   #Marco el vértice como no visitado
        self.predecesor = None   #Defino el vértice sin predecesor
        self.peso = float('inf')   #Defino el peso para llegar al vértice en infinito

    def add_adyacente(self, v, p ):   # Función para agregar los vertices que son adyacentes a v con sus respectivos pesos p
        if v not in self.adyacentes:
            self.adyacentes.append([v,p])

In [24]:
'''Clase que define el grafo'''
class Graph:
    
    def __init__(self):   #Defino el constructor de la clase Graph
        self.vertices = {}   #Instancio el diccionario que usará la clase

    def add_vertice(self, id):  # Función para agregar el vértice al {}
        if id not in self.vertices:  #Verifico que el id no esté en {} para poder agregarlo y que no hayan repetidos
            self.vertices[id] = Vertice(id)

    def add_arista(self, v1, v2, p):   #Función para agregar una arista entre dos nodos v1,v2 y su peso p
        if v1 in self.vertices and v2 in self.vertices:   #verifico que los dos vertices existan
            self.vertices[v1].add_adyacente(v2,p)   #añado la arista entre v1 y v2 con peso p
            #self.vertices[v2].add_adyacente(v1,p)  
    
    def imprimir(self): #Función para imprimir los valores finales de la gráfica
        for v in self.vertices:
            print("El peso para llegar al vértice " , v , " es " , self.vertices[v].peso , " llegando desde " , self.vertices[v].predecesor)

    def camino(self, v1, v2): #Función para imprimir el camino a seguir y su costo
        camino = []
        actual = v2
        while actual != None:
            camino.insert(0, actual)
            actual = self.vertices[actual].predecesor
        return [camino, self.vertices[v2].peso]

    def minimo(self, lista): # Función para elegir el siguiente vértice que tena el menor peso para usar en el algoritmo Dijkstra
        if len(lista) > 0:
            m = self.vertices[lista[0]].peso
            v = lista[0]
            for e in lista:
                if m > self.vertices[e].peso:
                    m = self.vertices[e].peso
                    v = e
            return v
    
    def dijkstra(self, inicio):  #Algoritmo para determinar el camino más corto
        if inicio in self.vertices: #Verifico que el vértice en que voy a iniciar exista
            self.vertices[inicio].peso = 0  #Defino su peso en 0
            actual = inicio  #Lo hago el vértice actual
            NoVisitados = []   #Instancio la lista de nodos Novisitados vacía

            for v in self.vertices: #Con este for recorro cada vértice del grafo
                if v != inicio: # Tomo todos los vértices diferentes del que va a ser el inicio
                    self.vertices[v].peso = float('inf')   # Defino su peso como infinito
                self.vertices[v].predecesor = None   #Defino su predecesor como None
                NoVisitados.append(v)   #Agrego el vértice a la lista NoVisitados

            while len(NoVisitados) > 0:  #Mientras que la lista de nodos no esté vacía 
                for vecino in self.vertices[actual].adyacentes:  #Para cada vértice adyacente al actual 
                    if self.vertices[vecino[0]].visitado == False: # Si el adyacente no ha sido visitado
                        if self.vertices[actual].peso + vecino[1] < self.vertices[vecino[0]].peso: #Verifico si el peso del actual + el de la arista es menor al peso del vértice
                            self.vertices[vecino[0]].peso = self.vertices[actual].peso + vecino[1] #Actualizo el nuevo peso del vértice adyacente
                            self.vertices[vecino[0]].predecesor = actual   #Hago el predecesor del vértice adyacente = al vértice actual

                self.vertices[actual].visitado = True #Una vez tomados todos los nodos adyacentes, marco el nodo actual como visitado
                NoVisitados.remove(actual)   #Lo elimino de la lista de NoVisitados

                actual= self.minimo(NoVisitados)  #Hago que el vértice actual sea el que esté en la lista NoVisitados y tenga el menor peso 
                
        else:
            return False   #Return si el nodo de inicio no existe


In [25]:
g1=Graph() # Instancio a g como un objeto de la clase Graph
g2=Graph()
g3=Graph()
for llave in origenes:   #Para cada llave
    g1.add_vertice(llave)   #La agrego como un nuevo vértice
    g2.add_vertice(llave)
    g3.add_vertice(llave)

for llave in origenes:   #Para cada llave
    if len(diccionario1[llave]) != 0:   #Si la llave tiene un valor(está conectada con más vértices)
        for i in range(len(diccionario1[llave][0])):
            for valor in diccionario1[llave][0]:   #Para cada vértice con el que está conectado
                g1.add_arista(llave,diccionario1[llave][0][i],diccionario1[llave][1][i])   #Agrego una arista entre la llave y el vértice con su respectivo peso
                g2.add_arista(llave,diccionario2[llave][0][i],diccionario2[llave][1][i])
                g3.add_arista(llave,diccionario3[llave][0][i],diccionario3[llave][1][i])

In [26]:
print("La ruta #1 más rápida por Dijkstra junto a su costo es: ")
g1.dijkstra("(-75.5948492, 6.2686223)")
a=g1.camino("(-75.5948492, 6.2686223)","(-75.592992, 6.2733618)")
print(a)

print("La ruta #2 más rápida por Dijkstra junto a su costo es: ")
g2.dijkstra("(-75.5948492, 6.2686223)")
b=g2.camino("(-75.5948492, 6.2686223)","(-75.592992, 6.2733618)")
print(b)

print("La ruta #3 más rápida por Dijkstra junto a su costo es: ")
g3.dijkstra("(-75.5948492, 6.2686223)")
c=g3.camino("(-75.5948492, 6.2686223)","(-75.592992, 6.2733618)")
print(c)


La ruta #1 más rápida por Dijkstra junto a su costo es: 
[['(-75.5948492, 6.2686223)', '(-75.5947225, 6.2691892)', '(-75.5946708, 6.2697116)', '(-75.594662, 6.2702315)', '(-75.5951151, 6.2703279)', '(-75.5943494, 6.2715022)', '(-75.5937591, 6.2720736)', '(-75.5932909, 6.272827)', '(-75.5930504, 6.2732544)', '(-75.5930251, 6.2732994)', '(-75.592992, 6.2733618)'], 16742.17149683342]
La ruta #2 más rápida por Dijkstra junto a su costo es: 
[['(-75.5948492, 6.2686223)', '(-75.5947225, 6.2691892)', '(-75.5946708, 6.2697116)', '(-75.594662, 6.2702315)', '(-75.5951151, 6.2703279)', '(-75.5943494, 6.2715022)', '(-75.5937591, 6.2720736)', '(-75.5932909, 6.272827)', '(-75.5930504, 6.2732544)', '(-75.5930251, 6.2732994)', '(-75.592992, 6.2733618)'], 1515.4426448846946]
La ruta #3 más rápida por Dijkstra junto a su costo es: 
[['(-75.5948492, 6.2686223)', '(-75.5947225, 6.2691892)', '(-75.5946708, 6.2697116)', '(-75.594662, 6.2702315)', '(-75.5951151, 6.2703279)', '(-75.5943494, 6.2715022)', '(-75

In [79]:
longitud=0
for element in (a[0]):
    longitud+=int(df.query('origin==@element').length.sum())

print('La longitud del trayecto es:',longitud)

La longitud del trayecto es: 1208


In [73]:
import folium 
medellin=folium.Map(location=[6.244864546094932, -75.57111493876913],zoom_start=11.5)
t=((-75.5948492, 6.2686223),(-75.5947225, 6.2691892),(-75.5946708, 6.2697116),(-75.594662, 6.2702315),(-75.5951151, 6.2703279),(-75.5943494, 6.2715022),(-75.5937591, 6.2720736),(-75.5932909, 6.272827),(-75.5930504, 6.2732544),(-75.5930251, 6.2732994),(-75.592992, 6.2733618))
for i in range(len(t)):
    folium.Marker(location=[t[i][1],t[i][0]],popup=i+1).add_to(medellin)

medellin